In [1]:
%load_ext autoreload

In [2]:
import os
import sys
import re
from argparse import Namespace

sys.path.append('../src-py/')

os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
%autoreload

from utils import *
from project_debater_api import *
from mt_bart_v2 import *

In [4]:
import torch
import json

import nltk
import numpy as np
import pandas as pd

from pathlib import Path
from datasets import load_dataset, load_metric, Dataset

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartTokenizer, BartForConditionalGeneration

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
device

device(type='cuda')

In [8]:
ceph_dir = '/home/sile2804/data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation'
local_home_dir = '../data'

In [9]:
#Evalute ROUGE scores
gen_kwargs = {
    "do_sample": False, 
    "max_length":200,
    "top_p":0.95, 
    "top_k":50,
    "num_beams":1
}

def generate_ds_attacks(ds, model, tokenizer, premises_clm, conclusion_clm, gen_kwargs, skip_special_tokens=True, batch_size=8):
    ds = ds.map(lambda x :preprocess_function(x, tokenizer, premises_clm, 'counter', conclusion_clm=conclusion_clm), batched=True)
    ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    dataloader = torch.utils.data.DataLoader(ds, batch_size=batch_size)
    attacks = generate_counters(model, tokenizer, dataloader, gen_kwargs, skip_special_tokens=skip_special_tokens)
    
    return attacks

#### Load Testing data:

##### Loading validation data from Reddit:

In [10]:
valid_df = pd.read_pickle(ceph_dir + '/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_with_targets.pkl')

In [11]:
#drop duplicate posts
unique_valid_posts = valid_df.drop_duplicates('post_id')

In [12]:
valid_ds = Dataset.from_pandas(unique_valid_posts)
tmp_ds = valid_ds.train_test_split(0.9)
sample_valid_ds = tmp_ds['test']

#saving the sample
reddit_sample_valid_ds = sample_valid_ds.flatten_indices()
reddit_sample_valid_ds.save_to_disk('../data/sample_valid_ds')

  0%|          | 0/3 [00:00<?, ?ba/s]

In [13]:
#loading the random sample of validation dataset with the auto-generated conclusion
reddit_sample_valid_ds = Dataset.load_from_disk('../data/sample_valid_ds/')

In [14]:
len(reddit_sample_valid_ds)

2278

##### Loading validation data from Kialo:

In [29]:
valid_ds = Dataset.from_pandas(pd.read_pickle(ceph_dir + '/kialo_data/kialo_valid_df.pkl'))

In [10]:
tmp_ds = valid_ds.train_test_split(0.5)
sample_valid_ds = tmp_ds['test']

#saving the sample
sample_valid_ds = sample_valid_ds.flatten_indices()
sample_valid_ds.save_to_disk('../data/kialo_sample_valid_ds')

  0%|          | 0/3 [00:00<?, ?ba/s]

In [9]:
sample_valid_ds = Dataset.load_from_disk('../data/kialo_sample_valid_ds')

### Evaluating on Kialo:

In [12]:
#KIALO models
masked_conclusion_kialo_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/masked-conclusion-kialo-model/').to(device)
masked_conclusion_kialo_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/masked-conclusion-kialo-model/')
known_conclusion_kialo_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/known-conclusion-kialo-model/').to(device)
known_conclusion_kialo_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/known-conclusion-kialo-model/')

#masked_conclusion_kialo_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/masked-conclusion-kialo-unique-model/').to(device)
#known_conclusion_kialo_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/known-conclusion-kialo-unique-model/').to(device)

In [15]:
#known_conc_attacks = generate_ds_attacks(sample_valid_ds, known_conclusion_kialo_model, known_conclusion_kialo_tokenizer, 'premises', 'conclusion_text', gen_kwargs)
#masked_conc_attacks = generate_ds_attacks(sample_valid_ds, masked_conclusion_kialo_model, masked_conclusion_kialo_tokenizer, 'premises', None, gen_kwargs)
#auto_conc_attacks = generate_ds_attacks(sample_valid_ds, known_conclusion_model, tokenizer, 'masked_premises', 'auto_conclusion', gen_kwargs)

In [12]:
kialo_pred_df = pd.DataFrame(list(zip(sample_valid_ds['conclusion_text'], sample_valid_ds['premises'], 
                                   sample_valid_ds['counter'], known_conc_attacks, masked_conc_attacks)), 
                      columns=['conclusion', 'premises', 'gt', 'known_conc_attacks', 'masked_conc_attacks'])

kialo_pred_df['argument'] = kialo_pred_df.apply(lambda row: row['conclusion'] + ' : ' + ' '.join(row['premises']), axis=1)
kialo_pred_df['premises'] = kialo_pred_df['premises'].apply(lambda x: ' '.join(x))

### Generate predictions on Reddit:

In [15]:
# Loading Reddit models

#masked_conclusion_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/extended_tokenizer')
#masked_conclusion_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/masked-conclusion-bart-model/').to(device)

known_conclusion_model = BartForConditionalGeneration.from_pretrained(local_home_dir  + '/output/known-conclusion-bart-model').to(device)
known_conclusion_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/extended_tokenizer')

pred_conclusion_model = BartForConditionalGeneration.from_pretrained(local_home_dir  + '/output/pred-conclusion-bart-model/').to(device)
pred_conclusion_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/pred-conclusion-bart-model/')

join_model_tokenizer = BartTokenizer.from_pretrained('../data/output/joint-con-counter-bart-model-no-attention/')
#join_model  = BartModelV2.from_pretrained('../data/output/joint-con-counter-bart-model-no-attention-finetuned/0-1-1-0/', conc_loss_weight = 0.1, counter_loss_weight=1.0, attention_to_conc=False, conc_decoder=True).to(device)
join_model  = BartModelV2.from_pretrained('../data/output/joint-con-counter-bart-model-no-attention-finetuned/dynamic-weight', compute_dynamic_weights=True, conc_decoder=True).to(device)

In [26]:
joint_attacks = generate_ds_attacks(reddit_sample_valid_ds, join_model, join_model_tokenizer, 'masked_premises', None, gen_kwargs)
known_conc_attacks  = generate_ds_attacks(reddit_sample_valid_ds, known_conclusion_model, known_conclusion_tokenizer, 'masked_premises', 'title', gen_kwargs)
bart_conc_attacks   = generate_ds_attacks(reddit_sample_valid_ds, known_conclusion_model, known_conclusion_tokenizer, 'masked_premises', 'bart_conclusion', gen_kwargs)
masked_conc_attacks = generate_ds_attacks(reddit_sample_valid_ds, known_conclusion_model, known_conclusion_tokenizer, 'masked_premises', None, gen_kwargs)
joint_conc_baseline_attacks  = generate_ds_attacks(reddit_sample_valid_ds, pred_conclusion_model, pred_conclusion_tokenizer, 'masked_premises', None, gen_kwargs, skip_special_tokens=False)
#This model would predict the conclusion and then the counter in one sequence.

ProConClient: 100%|██████████| 2172/2172 [13:07:21<00:00, 21.75s/it]


  0%|          | 0/3 [00:00<?, ?ba/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-a75c0a296a25>", line 1, in <module>
    joint_attacks = generate_ds_attacks(reddit_sample_valid_ds, join_model, join_model_tokenizer, 'masked_premises', None, gen_kwargs)
  File "<ipython-input-26-c90060ca9cd2>", line 11, in generate_ds_attacks
    ds = ds.map(lambda x :preprocess_function(x, tokenizer, premises_clm, 'counter', conclusion_clm=conclusion_clm), batched=True)
  File "/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py", line 1679, in map
    desc=desc,
  File "/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py", line 185, in wrapper
    out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/datasets/fingerprint.py", line 397, in wrapper
    out = func(self, *args, **kwarg

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-a75c0a296a25>", line 1, in <module>
    joint_attacks = generate_ds_attacks(reddit_sample_valid_ds, join_model, join_model_tokenizer, 'masked_premises', None, gen_kwargs)
  File "<ipython-input-26-c90060ca9cd2>", line 11, in generate_ds_attacks
    ds = ds.map(lambda x :preprocess_function(x, tokenizer, premises_clm, 'counter', conclusion_clm=conclusion_clm), batched=True)
  File "/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py", line 1679, in map
    desc=desc,
  File "/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py", line 185, in wrapper
    out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/datasets/fingerprint.py", line 397, in wrapper
    out = func(self, *args, **kwarg

TypeError: object of type 'NoneType' has no len()

In [18]:
reddit_pred_df = pd.DataFrame(list(zip(
                                       reddit_sample_valid_ds['post_id'],
                                       reddit_sample_valid_ds['title'], 
                                       reddit_sample_valid_ds['conclusion_targets'],
                                       reddit_sample_valid_ds['conclusion_stance'],
                                       reddit_sample_valid_ds['bart_conclusion'], 
                                       reddit_sample_valid_ds['masked_premises'],
                                       reddit_sample_valid_ds['counter'], 
                                       known_conc_attacks, masked_conc_attacks, 
                                       bart_conc_attacks, joint_attacks, joint_conc_baseline_attacks)), 
                columns=['post_id', 'conclusion', 'conclusion_target', 'conclusion_stance', 'bart_conclusion', 'premises', 'gt_attack', 'known_conc_attacks', 
                         'masked_conc_attacks', 'bart_conc_attacks', 'joint_conc_attacks', 'joint_conc_baseline_attacks'])

reddit_pred_df['argument'] = reddit_pred_df.apply(lambda row: row['conclusion'] + ' : ' + ' '.join(row['premises']), axis=1)
reddit_pred_df['premises'] = reddit_pred_df['premises'].apply(lambda x: ' '.join(x))

#process the jointly generated conclusion and counter
reddit_pred_df['joint_conc_baseline'] = reddit_pred_df['joint_conc_baseline_attacks'].apply (lambda x: x.split('<counter>')[0])
reddit_pred_df['joint_conc_baseline_attacks'] = reddit_pred_df['joint_conc_baseline_attacks'].apply (lambda x: x.split('<counter>')[1] if '<counter>' in x else x)
reddit_pred_df['joint_conc_baseline'] = reddit_pred_df['joint_conc_baseline'].apply (lambda x: re.sub('<s>|</s>|<conclusion>|<counter>|<pad>', '', x).strip())
reddit_pred_df['joint_conc_baseline_attacks'] = reddit_pred_df['joint_conc_baseline_attacks'].apply (lambda x: re.sub('<s>|</s>|<conclusion>|<counter>|<pad>', '', x).strip())

#save created pred_df
reddit_pred_df.to_pickle('../data/output/reddit_pred_df-nosample.pkl')

### Evaluate generated predictions:

In [16]:
reddit_pred_df = pd.read_pickle('../data/output/reddit_pred_df-nosample.pkl')

In [17]:
reddit_pred_df[['argument', 'conclusion' , 'joint_conc_baseline', 'masked_conc_attacks', 'known_conc_attacks', 'joint_conc_baseline_attacks', 'joint_conc_attacks']].sample(10).head(n=10)

argument  \
1682                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [18]:
#collect references
arg_counters = valid_df.groupby('post_id').agg({
    'counter': lambda x: [' '.join(c) for c in x]
}).reset_index()
arg_counters = pd.Series(arg_counters.counter.values, index=arg_counters.post_id).to_dict()

reddit_pred_df['gt_attack'] = reddit_pred_df['gt_attack'].apply(lambda x: str(x))
reddit_pred_df['all_counters'] = reddit_pred_df['post_id'].apply(lambda x: arg_counters[x])

In [19]:
reddit_pred_df = reddit_pred_df[reddit_pred_df.all_counters.map(len) > 0]

In [20]:
masked_conc_eval = evaluate_gen_attacks(reddit_pred_df['masked_conc_attacks'].tolist(), reddit_pred_df['all_counters'].tolist())
known_conc_eval = evaluate_gen_attacks(reddit_pred_df['known_conc_attacks'].tolist(), reddit_pred_df['all_counters'].tolist())
bart_conc_eval = evaluate_gen_attacks(reddit_pred_df['bart_conc_attacks'].tolist(), reddit_pred_df['all_counters'].tolist())
pred_conc_eval = evaluate_gen_attacks(reddit_pred_df['joint_conc_baseline_attacks'].tolist(), reddit_pred_df['all_counters'].tolist())
joint_conc_eval = evaluate_gen_attacks(reddit_pred_df['joint_conc_attacks'].tolist(), reddit_pred_df['all_counters'].tolist())

##### Test stance correcteness:

In [21]:
filtered_reddit_pred_df = reddit_pred_df[pd.notna(reddit_pred_df.conclusion_target)]

In [22]:
#compute the stance of the generated counters towards the conclusion target
filtered_reddit_pred_df['masked_conc_stances'] = get_stances(filtered_reddit_pred_df.conclusion_target.tolist(), filtered_reddit_pred_df.masked_conc_attacks.tolist())
filtered_reddit_pred_df['known_conc_stances']  = get_stances(filtered_reddit_pred_df.conclusion_target.tolist(), filtered_reddit_pred_df.known_conc_attacks.tolist())
filtered_reddit_pred_df['bart_conc_stances']   = get_stances(filtered_reddit_pred_df.conclusion_target.tolist(), filtered_reddit_pred_df.bart_conc_attacks.tolist())
filtered_reddit_pred_df['pred_conc_stances']  = get_stances(filtered_reddit_pred_df.conclusion_target.tolist(), filtered_reddit_pred_df.joint_conc_baseline_attacks.tolist())
filtered_reddit_pred_df['joint_conc_stances']  = get_stances(filtered_reddit_pred_df.conclusion_target.tolist(), filtered_reddit_pred_df.joint_conc_attacks.tolist())

ProConClient: 100%|██████████| 2172/2172 [01:07<00:00, 32.13it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

ProConClient: 100%|██████████| 2172/2172 [00:42<00:00, 43.94it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


ProConClient: 100%|██████████| 2172/2172 [00:54<00:00, 43.94it/s]

ProConClien

In [23]:
#compute the distance between the conclusion stance and the attack stance: the bigger the distance the better...
masked_conc_stance_score1 = round(np.mean([abs(x[0] - x[1]) for x in zip(filtered_reddit_pred_df.masked_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())]), 2)
known_conc_stance_score1 = round(np.mean([abs(x[0] - x[1]) for x in zip(filtered_reddit_pred_df.known_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())]), 2)
bart_conc_stance_score1 = round(np.mean([abs(x[0] - x[1]) for x in zip(filtered_reddit_pred_df.bart_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())]), 2)
pred_conc_stance_score1 = round(np.mean([abs(x[0] - x[1]) for x in zip(filtered_reddit_pred_df.pred_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())]), 2)
joint_conc_stance_score1 = round(np.mean([abs(x[0] - x[1]) for x in zip(filtered_reddit_pred_df.joint_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())]), 2)

#check if the two stances are contradicotry
masked_conc_stance_score2 = round(sum([int(x[0] * x[1] < 0) for x in zip(filtered_reddit_pred_df.masked_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())])/len(filtered_reddit_pred_df), 2)
known_conc_stance_score2  = round(sum([int(x[0] * x[1] < 0) for x in zip(filtered_reddit_pred_df.known_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())])/len(filtered_reddit_pred_df), 2)
bart_conc_stance_score2   = round(sum([int(x[0] * x[1] < 0) for x in zip(filtered_reddit_pred_df.bart_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())])/len(filtered_reddit_pred_df), 2)
pred_conc_stance_score2  = round(sum([int(x[0] * x[1] < 0) for x in zip(filtered_reddit_pred_df.pred_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())])/len(filtered_reddit_pred_df), 2)
joint_conc_stance_score2  = round(sum([int(x[0] * x[1] < 0) for x in zip(filtered_reddit_pred_df.joint_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())])/len(filtered_reddit_pred_df), 2)

In [24]:
from tabulate import tabulate

print(tabulate([['Masked Conclusion'] + [round(masked_conc_eval['bleu'], 2), masked_conc_eval['bert-fscore'], masked_conc_stance_score1, masked_conc_stance_score2],
                ['BART Conclusion'] + [round(bart_conc_eval['bleu'], 2), bart_conc_eval['bert-fscore'], bart_conc_stance_score1, bart_conc_stance_score2],
                ['Joint Prediction (baseline)'] + [round(pred_conc_eval['bleu'], 2), pred_conc_eval['bert-fscore'], joint_conc_stance_score1, joint_conc_stance_score2],
                ['Joint Prediction'] + [round(joint_conc_eval['bleu'], 2), joint_conc_eval['bert-fscore'], joint_conc_stance_score1, joint_conc_stance_score2],
                ['Known Conclusion'] + [round(known_conc_eval['bleu'], 2), known_conc_eval['bert-fscore'], known_conc_stance_score1, known_conc_stance_score2],
    ], headers=['bleu', 'bert-f1score', 'stance-score (diff)', 'stance-score (agreement)']))

                               bleu    bert-f1score    stance-score (diff)    stance-score (agreement)
---------------------------  ------  --------------  ---------------------  --------------------------
Masked Conclusion              0.13            0.11                   0.85                        0.55
BART Conclusion                0.13            0.12                   0.84                        0.54
Joint Prediction (baseline)    0.15            0.15                   0.86                        0.55
Joint Prediction               0.15            0.13                   0.86                        0.55
Known Conclusion               0.14            0.13                   0.87                        0.55


In [25]:
from tabulate import tabulate

print(tabulate([['Masked Conclusion'] + [round(masked_conc_eval['bleu'], 2), masked_conc_eval['bert-fscore'], masked_conc_stance_score1, masked_conc_stance_score2],
                ['BART Conclusion'] + [round(bart_conc_eval['bleu'], 2), bart_conc_eval['bert-fscore'], bart_conc_stance_score1, bart_conc_stance_score2],
                ['Joint Prediction (baseline)'] + [round(pred_conc_eval['bleu'], 2), pred_conc_eval['bert-fscore'], joint_conc_stance_score1, joint_conc_stance_score2],
                ['Joint Prediction'] + [round(joint_conc_eval['bleu'], 2), joint_conc_eval['bert-fscore'], joint_conc_stance_score1, joint_conc_stance_score2],
                ['Known Conclusion'] + [round(known_conc_eval['bleu'], 2), known_conc_eval['bert-fscore'], known_conc_stance_score1, known_conc_stance_score2],
    ], headers=['bleu', 'bert-f1score', 'stance-score (diff)', 'stance-score (agreement)']))

                               bleu    bert-f1score    stance-score (diff)    stance-score (agreement)
---------------------------  ------  --------------  ---------------------  --------------------------
Masked Conclusion              0.13            0.11                   0.85                        0.55
BART Conclusion                0.13            0.12                   0.84                        0.54
Joint Prediction (baseline)    0.15            0.15                   0.78                        0.54
Joint Prediction               0.12            0.09                   0.78                        0.54
Known Conclusion               0.14            0.13                   0.87                        0.55


##### Similarity to conclusion:

In [15]:
masked_conc_eval = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['masked_conc_attacks'].tolist())
known_conc_eval  = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['known_conc_attacks'].tolist())
#auto_conc_eval   = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['auto_conc_attacks'].tolist())
pred_conc_eval   = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['pred_conc_attacks'].tolist())
gt_attack_conc_eval = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['gt'].tolist())

In [17]:
from tabulate import tabulate

print(tabulate([['masked'] + [round(masked_conc_eval['bleu'], 2), masked_conc_eval['bert-fscore']],
                #['auto'] + list(auto_conc_eval.values()),
                ['pred'] + [round(pred_conc_eval['bleu'], 2), pred_conc_eval['bert-fscore']],
                ['known'] + [round(known_conc_eval['bleu'], 2), known_conc_eval['bert-fscore']],
                ['gt_attack'] + [round(gt_attack_conc_eval['bleu'], 2), gt_attack_conc_eval['bert-fscore']]
    ], headers=['bleu', 'bert-f1score']))

             bleu    bert-f1score
---------  ------  --------------
masked       0               0.16
pred         0.01            0.18
known        0.03            0.33
gt_attack    0              -0.03
